In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,date

In [2]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [45]:
def source_predict_state_2(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name","model_run_datetime","create_time",
    "media_id","game_id","platform","data_win","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
    "create_role_retain_1d","create_role_pay_rate","create_role_pay_num_cum","learning_type","learning_time_dt",
    "learning_time_hr","deep_bid_type"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])

    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    data.sort_values(by='data_win', inplace=True)
    data.drop_duplicates(subset=['channel_id', 'source_id', 'source_run_date_amount', 'create_role_num'], keep='first'
                         , inplace=True)
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[(data['source_run_date_amount'] >= 300) | (data['data_win'] == 1)]
    data_ignore = data.drop(data_not_ignore.index)

    data_win_1 = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 1)]
    data_win_2 = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 1)]
    data_win_3 = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 1)]

    data_win_1_ios = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 2)]
    data_win_2_ios = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 2)]
    data_win_3_ios = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 2)]

    # win=1预判
    result_win1 = pd.DataFrame()
    temp_win1 = pd.DataFrame()

    # data_win_1中再分deep_bid_type
    data_win_1_pre_action = data_win_1[data_win_1['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1 = data_win_1[data_win_1['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_pre_action.shape[0] != 0:
        data_win_1_pre_action['label'] = data_win_1_pre_action.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                                                 x.create_role_roi < 0.018 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 8000) | (x.create_role_cost > 220) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_pre_action[(data_win_1_pre_action['label'] == 1) | (data_win_1_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_pre_action[data_win_1_pre_action['label'] == 2])

    if data_win_1.shape[0] != 0:
        data_win_1['label'] = data_win_1.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                           x.create_role_roi <= 0.018 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 8000) | (x.create_role_cost > 140) else 2))), axis=1)

        result_win1 = result_win1.append(data_win_1[(data_win_1['label'] == 1) | (data_win_1['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1[data_win_1['label'] == 2])

    data_win_1_ios_pre_action = data_win_1_ios[data_win_1_ios['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1_ios = data_win_1_ios[data_win_1_ios['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_ios_pre_action.shape[0] != 0:
        data_win_1_ios_pre_action['label'] = data_win_1_ios_pre_action.apply(
            lambda x: 1 if x.create_role_num == 0 else (1 if
                                                        x.create_role_roi <= 0.016 else (
                0 if (x.create_role_pay_num > 1) & (
                        x.create_role_roi >= 0.018) & (x.source_run_date_amount >= 500) else (
                    0 if (x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else 1 if
                    (x.create_role_pay_cost > 12000) | (x.create_role_cost > 300) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios_pre_action[
                (data_win_1_ios_pre_action['label'] == 1) | (data_win_1_ios_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios_pre_action[data_win_1_ios_pre_action['label'] == 2])

    if data_win_1_ios.shape[0] != 0:
        data_win_1_ios['label'] = data_win_1_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                                   x.create_role_roi <= 0.016 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.018) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 12000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios[(data_win_1_ios['label'] == 1) | (data_win_1_ios['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios[data_win_1_ios['label'] == 2])

    # win=1 模型预测
    if temp_win1.shape[0] != 0:
        temp_win1['label'] = gbdt_b_win1.predict(temp_win1[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate']])
    result_win1_data = pd.concat([result_win1, temp_win1], axis=0)

    # win=2预判
    result_win2 = pd.DataFrame()
    temp_win2 = pd.DataFrame()
    if data_win_2.shape[0] != 0:
        data_win_2['label'] = data_win_2.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                           x.create_role_roi <= 0.033 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.035) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.038) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 7000) | (x.create_role_cost > 150) else 2))), axis=1)

        result_win2 = result_win2.append(data_win_2[(data_win_2['label'] == 1) | (data_win_2['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2[data_win_2['label'] == 2])

    if data_win_2_ios.shape[0] != 0:
        data_win_2_ios['label'] = data_win_2_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                                   x.create_role_roi <= 0.033 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.035) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.038) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 10000) | (x.create_role_cost > 250) else 2))), axis=1)

        result_win2 = result_win2.append(
            data_win_2_ios[(data_win_2_ios['label'] == 1) | (data_win_2_ios['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2_ios[data_win_2_ios['label'] == 2])

    # win=2 模型预测
    if temp_win2.shape[0] != 0:
        temp_win2['label'] = gbdt_b_win2.predict(temp_win2[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win2_data = pd.concat([result_win2, temp_win2], axis=0)

    # win=3预判
    result_win3 = pd.DataFrame()
    temp_win3 = pd.DataFrame()
    if data_win_3.shape[0] != 0:
        data_win_3['label'] = data_win_3.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                           x.create_role_roi <= 0.05 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.06) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 6000) | (x.create_role_cost > 120) else 2))), axis=1)

        result_win3 = result_win3.append(data_win_3[(data_win_3['label'] == 1) | (data_win_3['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3[data_win_3['label'] == 2])

    if data_win_3_ios.shape[0] != 0:
        data_win_3_ios['label'] = data_win_3_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
                                                                                                   x.create_role_roi <= 0.05 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.06) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 9000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win3 = result_win3.append(
            data_win_3_ios[(data_win_3_ios['label'] == 1) | (data_win_3_ios['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3_ios[data_win_3_ios['label'] == 2])

    # win=3 模型预测
    if temp_win3.shape[0] != 0:
        temp_win3['label'] = gbdt_b_win3.predict(temp_win3[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win3_data = pd.concat([result_win3, temp_win3], axis=0)

    result = pd.concat([result_win1_data, result_win2_data], axis=0)

    # 结果输出
    if result.shape[0] != 0:
        # 1\2有一个为开，则开
        if (result[result['data_win'] == 2].shape[0] != 0) & (result[result['data_win'] == 1].shape[0] != 0):
            result_1_2 = result[(result['data_win'] == 1) | (result['data_win'] == 2)]
            result_1_2_label = result_1_2[['channel_id', 'source_id', 'model_run_datetime', 'data_win', 'label']]
            result_1_2_label['data_win'] = result_1_2_label['data_win'].astype(int)
            result_1_2_piv = pd.pivot_table(result_1_2_label, index=['channel_id', 'source_id', 'model_run_datetime'],
                                            columns='data_win')
            result_1_2_piv.columns = result_1_2_piv.columns.droplevel()
            result_1_2_piv = result_1_2_piv.rename(columns={1: 'label_1', 2: 'label_2'})

            result_1_2_piv = result_1_2_piv.reset_index()
            result_1_2_piv['label'] = result_1_2_piv.apply(
                lambda x: 0 if x.label_1 == 0 else (0 if x.label_2 == 0 else 1),
                axis=1)
            result_1_2 = result_1_2.drop('label', axis=1)
#             result_1_2 = result_1_2[result_1_2['data_win'] == 1]
            result_1_2 = result_1_2.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')
        
            result_1_2_piv.drop(['label_1', 'label_2'], axis=1, inplace=True)
            source_predict = pd.merge(result_1_2, result_1_2_piv, on=['channel_id', 'source_id', 'model_run_datetime'],
                                      how='left')
        else:
            source_predict = result

    else:
        source_predict = pd.DataFrame(
            columns=["channel_id", "source_id", "plan_name", "model_run_datetime", "create_time",
                     "media_id", "game_id", "platform", "data_win", "source_run_date_amount", "create_role_num",
                     "create_role_cost", "create_role_pay_num", "create_role_pay_cost", "create_role_pay_sum",
                     "create_role_roi",
                     "create_role_retain_1d", "create_role_pay_rate", "create_role_pay_num_cum", "learning_type",
                     "learning_time_dt",
                     "learning_time_hr", "deep_bid_type", "label"])

    if data_ignore.shape[0] != 0:
        data_ignore['label'] = 1
        data_ignore.sort_values('data_win', ascending=False, inplace=True)
        data_ignore = data_ignore.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')
        data_ignore['data_win'] = -1

    source_predict = pd.concat([source_predict, data_ignore], axis=0)
    source_predict.sort_values('data_win', ascending=False, inplace=True)
    source_predict = source_predict.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

    source_predict['label'] = source_predict['label'].astype(int)
    # print(source_predict.columns.values.tolist())
    source_predict = source_predict.fillna('null')
    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [46]:
state_2_data = pd.DataFrame(data=[
           [20451,768534,"FW-幸存者-智模_0712_52156922","2021-07-14 00:00:00","2021-07-12 00:00:00",10,1001379,1,2,2782.18,41,67.8580488,2,1391.090000,238.00,0.0855444,0.0976,0.04878,2,'null','null','null',"BID_PER_ACTION"],
            [20451,768534,"FW-幸存者-智模_0712_52156922","2021-07-14 00:00:00","2021-07-12 00:00:00",10,1001379,1,3,2782.18,41,67.8580488,2,1391.090000,238.00,0.0855444,0.0976,0.04878,2,'null','null','null',"BID_PER_ACTION"],
            [20451,768534,"FW-幸存者-智模_0712_52156922","2021-07-14 00:00:00","2021-07-12 00:00:00",10,1001379,1,7,2782.18,41,67.8580488,2,1391.090000,238.00,0.0855444,0.0976,0.04878,2,'null','null','null',"BID_PER_ACTION"],
[20451,768315,"JIN-幸存者-智模_0712_161283711","2021-07-14 00:00:00","2021-07-12 00:00:00",10,1001379,1,1,3302.12,59,55.9681356,0,0.000000,0.00,0E-7,0.0847,0.00000,0,'null','null','null',"BID_PER_ACTION"],
[20451,768315,"JIN-幸存者-智模_0712_161283711","2021-07-14 00:00:00","2021-07-12 00:00:00",10,1001379,1,2,34562.12,455,55.9681356,0,0.000000,0.00,0E-7,0.0847,0.00000,0,'null','null','null',"BID_PER_ACTION"]],
            columns = [
            "channel_id",
            "source_id",
            "plan_name",
            "model_run_datetime",
            "create_time",
            "media_id",
            "game_id",
            "platform",
            "data_win",
            "source_run_date_amount",
            "create_role_num",
            "create_role_cost",
            "create_role_pay_num",
            "create_role_pay_cost",
            "create_role_pay_sum",
            "create_role_roi",
            "create_role_retain_1d",
            "create_role_pay_rate",
            "create_role_pay_num_cum",
            "learning_type",
            "learning_time_dt",
            "learning_time_hr",
            "deep_bid_type"])

In [47]:
jsondata = {"columns": state_2_data.columns,
           "data":state_2_data.values}
columns_2 = source_predict_state_2(jsondata)['columns']
data_2 = source_predict_state_2(jsondata)['list']
df2 = pd.DataFrame(data_2,columns=columns_2)
df2

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,data_win,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type,label
0,20451,768534,FW-幸存者-智模_0712_52156922,2021-07-14 00:00:00,2021-07-12 00:00:00,10,1001379,1,2,2782.18,41,67.858049,2,1391.09,238.0,0.085544,0.0976,0.04878,2,null,null,null,BID_PER_ACTION,0
1,20451,768315,JIN-幸存者-智模_0712_161283711,2021-07-14 00:00:00,2021-07-12 00:00:00,10,1001379,1,1,3302.12,59,55.968136,0,0.00,0.0,0.000000,0.0847,0.00000,0,null,null,null,BID_PER_ACTION,1


In [28]:
result_1_2 = pd.DataFrame({'channel_id':[1,1,2],'source_id':[2,2,4],'model_run_datetime':[3,3,3],'data_win':[1,2,2],'label':[0,1,0]})

In [29]:
result_1_2

,channel_id,source_id,model_run_datetime,data_win,label
0,1,2,3,1,0
1,1,2,3,2,1
2,2,4,3,2,0


In [30]:
result_1_2_label = result_1_2[['channel_id', 'source_id', 'model_run_datetime', 'data_win', 'label']]
result_1_2_label['data_win'] = result_1_2_label['data_win'].astype(int)

In [31]:
result_1_2_piv = pd.pivot_table(result_1_2_label, index=['channel_id', 'source_id', 'model_run_datetime'],
                                columns='data_win')

In [32]:
result_1_2_piv

label     
data_win                                    1    2
channel_id source_id model_run_datetime           
1          2         3                    0.0  1.0
2          4         3                    NaN  0.0

In [33]:
result_1_2_piv.columns = result_1_2_piv.columns.droplevel()
result_1_2_piv = result_1_2_piv.rename(columns={1: 'label_1', 2: 'label_2'})

In [34]:
result_1_2_piv

,,data_win,label_1,label_2
channel_id,source_id,model_run_datetime,,
1,2,3,0.0,1.0
2,4,3,NaN,0.0


In [35]:
result_1_2_piv = result_1_2_piv.reset_index()

In [36]:
result_1_2_piv

data_win,channel_id,source_id,model_run_datetime,label_1,label_2
0,1,2,3,0.0,1.0
1,2,4,3,NaN,0.0


In [37]:
result_1_2_piv['label'] = result_1_2_piv.apply(
    lambda x: 0 if x.label_1 == 0 else (0 if x.label_2 == 0 else 1),
    axis=1)

In [38]:
result_1_2_piv

data_win,channel_id,source_id,model_run_datetime,label_1,label_2,label
0,1,2,3,0.0,1.0,0
1,2,4,3,NaN,0.0,0


In [39]:
result_1_2 = result_1_2.drop('label', axis=1)

In [40]:
# result_1_2 = result_1_2[result_1_2['data_win'] == 1]
result_1_2 = result_1_2.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

In [41]:
result_1_2

,channel_id,source_id,model_run_datetime,data_win
0,1,2,3,1
2,2,4,3,2


In [42]:
result_1_2_piv.drop(['label_1', 'label_2'], axis=1, inplace=True)

In [43]:
source_predict = pd.merge(result_1_2, result_1_2_piv, on=['channel_id', 'source_id', 'model_run_datetime'],
                          how='left')

In [44]:
source_predict

,channel_id,source_id,model_run_datetime,data_win,label
0,1,2,3,1,0
1,2,4,3,2,0


In [ ]:
result_1_2_label = result_1_2[['channel_id', 'source_id', 'model_run_datetime', 'data_win', 'label']]
result_1_2_label['data_win'] = result_1_2_label['data_win'].astype(int)
result_1_2_piv = pd.pivot_table(result_1_2_label, index=['channel_id', 'source_id', 'model_run_datetime'],
                                columns='data_win')
result_1_2_piv.columns = result_1_2_piv.columns.droplevel()
result_1_2_piv = result_1_2_piv.rename(columns={1: 'label_1', 2: 'label_2'})

result_1_2_piv = result_1_2_piv.reset_index()
result_1_2_piv['label'] = result_1_2_piv.apply(
    lambda x: 0 if x.label_1 == 0 else (0 if x.label_2 == 0 else 1),
    axis=1)
result_1_2 = result_1_2.drop('label', axis=1)
result_1_2 = result_1_2[result_1_2['data_win'] == 1]
result_1_2_piv.drop(['label_1', 'label_2'], axis=1, inplace=True)
source_predict = pd.merge(result_1_2, result_1_2_piv, on=['channel_id', 'source_id', 'model_run_datetime'],
                          how='left')